# Testing the file workings

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

print("OpenAI Key:", os.getenv("OPENAI_API_KEY") is not None)
print("Claude Key:", os.getenv("ANTHROPIC_API_KEY") is not None)
print("Gemini Key:", os.getenv("GOOGLE_API_KEY") is not None)

OpenAI Key: True
Claude Key: True
Gemini Key: True


In [ ]:
import os
os.environ.pop("ANTHROPIC_API_KEY", None)

print("After removal:", repr(os.environ.get("ANTHROPIC_API_KEY")))

## API .env calls

In [ ]:
os.getenv("OPENAI_API_KEY")

In [ ]:
os.getenv("ANTHROPIC_API_KEY")

In [ ]:
os.getenv("GOOGLE_API_KEY")

## Basic LLM Calls

In [ ]:
import os
import asyncio

from models.openai_model import call_openai
from models.claude_model import call_claude
from models.gemini_model import call_gemini


OPENAI_KEY = os.getenv("OPENAI_API_KEY")
CLAUDE_KEY = os.getenv("ANTHROPIC_API_KEY")
GEMINI_KEY = os.getenv("GOOGLE_API_KEY")


async def test_all_models():

    prompt = 'Return ONLY valid JSON: {"status": "working"}'

    print("Testing OpenAI...")
    openai_result = await call_openai(prompt, api_key=OPENAI_KEY)
    print("OpenAI Result:", openai_result)

    print("\nTesting Claude...")
    claude_result = await call_claude(prompt, api_key=CLAUDE_KEY)
    print("Claude Result:", claude_result)

    print("\nTesting Gemini...")
    gemini_result = await call_gemini(prompt, api_key=GEMINI_KEY)
    print("Gemini Result:", gemini_result)

    return {
        "openai": openai_result,
        "claude": claude_result,
        "gemini": gemini_result
    }


results = await test_all_models()


In [ ]:
async def test_parallel():

    prompt = 'Return ONLY valid JSON: {"status": "parallel_working"}'

    tasks = [
        call_openai(prompt, api_key=OPENAI_KEY),
        call_claude(prompt, api_key=CLAUDE_KEY),
        call_gemini(prompt, api_key=GEMINI_KEY)
    ]

    results = await asyncio.gather(*tasks)

    print("OpenAI:", results[0])
    print("Claude:", results[1])
    print("Gemini:", results[2])

await test_parallel()


## Testing Segmentation

In [2]:
sample_contract="""
1. PAYMENT TERMS
Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.

2. INDEMNIFICATION
Vendor shall indemnify, defend, and hold harmless Client against any and all third-party claims arising from Vendor’s breach of this Agreement.

3. LIMITATION OF LIABILITY
In no event shall Vendor’s total liability exceed the total fees paid under this Agreement during the preceding twelve (12) months.

4. DATA SECURITY
Vendor shall implement commercially reasonable administrative, technical, and physical safeguards to protect Client Data from unauthorized access.

5. TERMINATION FOR CONVENIENCE
Client may terminate this Agreement at any time without cause upon thirty (30) days written notice.

6. CONFIDENTIALITY
Each party agrees to maintain the confidentiality of all non-public information disclosed under this Agreement.

7. FORCE MAJEURE
Neither party shall be liable for failure to perform due to events beyond reasonable control including natural disasters or governmental actions.

8. GOVERNING LAW
This Agreement shall be governed by and construed in accordance with the laws of the State of New York.

9. AUTO-RENEWAL
This Agreement shall automatically renew for successive one-year terms unless either party provides written notice at least sixty (60) days prior to expiration.

10. ASSIGNMENT
Vendor may not assign this Agreement without prior written consent of Client.


"""

In [3]:
from core.segmentation import segment_contract
clauses = await segment_contract(sample_contract)

print("Segmentation Output:")
print(clauses)

Segmentation Output:
[{'clause_id': 1, 'clause_heading': 'PAYMENT TERMS', 'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.'}, {'clause_id': 2, 'clause_heading': 'INDEMNIFICATION', 'clause_text': 'Vendor shall indemnify, defend, and hold harmless Client against any and all third-party claims arising from Vendor’s breach of this Agreement.'}, {'clause_id': 3, 'clause_heading': 'LIMITATION OF LIABILITY', 'clause_text': 'In no event shall Vendor’s total liability exceed the total fees paid under this Agreement during the preceding twelve (12) months.'}, {'clause_id': 4, 'clause_heading': 'DATA SECURITY', 'clause_text': 'Vendor shall implement commercially reasonable administrative, technical, and physical safeguards to protect Client Data from unauthorized access.'}, {'clause_id': 5, 'clause_heading': 'TERMINATION FOR CONVENIENCE', 'clause_text': 'Client may terminate this Agreement at any time 

In [7]:
from core.analysis import initial_analysis
from core.arbitration import arbitration
from core.review import review_round

clause_text = clauses[0]["clause_text"]

initial = await initial_analysis(clause_text)
reviewed = await review_round(clause_text, initial)

final = await arbitration(clause_text, reviewed)

print("Final Arbitration Output:")
print(final)


Final Arbitration Output:
{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.', 'golden_clause_detected': True, 'golden_clause_type': 'Payment', 'final_risk_score': 7.0, 'risk_level': 'High', 'business_risk_if_ignored': 'Ignoring this clause as-is exposes the Client to significant financial penalties in case of late payments. An interest rate of 2% per month (24% per annum) can quickly escalate outstanding debts, severely impacting cash flow and profitability, potentially leading to disputes and strained commercial relationships. While the 30-day term for undisputed invoices is standard, the punitive interest rate creates a substantial and unbalanced financial risk.', 'suggested_correction': 'Revise the clause to reduce the late payment interest rate to a more commercially reasonable and balanced level, such as 1% per month (12% per annum).', 'justification': "All models correctly identify thi

In [5]:
final

{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.',
 'golden_clause_detected': True,
 'golden_clause_type': 'Payment',
 'final_risk_score': 5.5,
 'risk_level': 'High',
 'business_risk_if_ignored': 'If this clause is ignored, the Client faces a significant financial burden for any late payments, as the 2% per month interest rate compounds to an annual rate of 24%. This rate is widely considered punitive rather than compensatory and could lead to rapid accumulation of debt. Such a high rate may also be challenged for enforceability in various jurisdictions, potentially leading to legal disputes and damage to the commercial relationship. It creates an imbalanced risk profile, disproportionately penalizing the Client for delayed payments.',
 'suggested_correction': "Reduce the late payment interest rate to a more commercially reasonable and compensatory level. A common revision would be to 1% pe

In [6]:
import time

async def test_multiple_clauses(clauses_subset):

    results = []

    start = time.time()

    for i, clause in enumerate(clauses_subset):
        print(f"\nProcessing clause {i+1}/{len(clauses_subset)}")

        initial = await initial_analysis(clause["clause_text"])
        reviewed = await review_round(clause["clause_text"], initial)
        final = await arbitration(clause["clause_text"], reviewed)

        results.append(final)

    end = time.time()

    print("\nTotal Time:", round(end - start, 2), "seconds")
    print("Average per clause:", round((end - start)/len(clauses_subset), 2), "seconds")

    return results


In [7]:
results = await test_multiple_clauses(clauses[:5])



Processing clause 1/5

Processing clause 2/5

Processing clause 3/5

Processing clause 4/5

Processing clause 5/5

Total Time: 212.07 seconds
Average per clause: 42.41 seconds


In [8]:
results

[{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.',
  'golden_clause_detected': True,
  'golden_clause_type': 'Payment',
  'final_risk_score': 6.8,
  'risk_level': 'High',
  'business_risk_if_ignored': 'If the client fails to pay invoices within the stipulated 30-day period, they will incur substantial and rapidly escalating late fees due to the high 2% per month interest rate (equivalent to 24% per annum). This can significantly increase the cost of doing business and lead to unforeseen financial liabilities, negatively impacting cash flow and profitability.',
  'suggested_correction': 'Consider negotiating the late payment interest rate down to a more standard commercial rate, such as 1% to 1.5% per month (12%-18% per annum), or tying it to a benchmark rate plus a reasonable margin (e.g., Prime + 2%).',
  'justification': "All models correctly identify the clause as a 'Payment' clause due

In [8]:
import time

async def test_sequential(clause_text):
    start = time.time()

    initial = await initial_analysis(clause_text)
    reviewed = await review_round(clause_text, initial)
    final = await arbitration(clause_text, reviewed)

    end = time.time()

    print("Sequential time:", round(end - start, 2), "seconds")
    return final


await test_sequential(clauses[0]["clause_text"])

Sequential time: 39.21 seconds


{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.',
 'golden_clause_detected': True,
 'golden_clause_type': 'Payment',
 'final_risk_score': 5.5,
 'risk_level': 'Medium',
 'business_risk_if_ignored': 'If this clause is ignored, the Client faces significant financial exposure due to an exceptionally high late payment interest rate (2% per month, equivalent to 24% per annum). This rate is likely above market standards and could lead to substantial and disproportionate penalties for even minor payment delays, creating an unbalanced contractual obligation and potential for disputes.',
 'suggested_correction': "Revise the late payment interest rate to a more commercially reasonable and balanced level, such as 1% per month (12% per annum), or a rate that aligns with statutory caps or prevailing industry standards in the relevant jurisdiction. For example: 'Late payments shall incur interest at 1% p

In [9]:
import asyncio
import os
from models.openai_model import call_openai
from models.claude_model import call_claude
from models.gemini_model import call_gemini
from config.prompts import ANALYSIS_PROMPT
from config.golden_clauses import GOLDEN_CLAUSES
from core.schemas import AnalysisOutput

async def parallel_initial_analysis(clause_text):

    prompt = ANALYSIS_PROMPT.format(
        clause_text=clause_text,
        golden_clauses=GOLDEN_CLAUSES
    )

    openai_key = os.getenv("OPENAI_API_KEY")
    claude_key = os.getenv("ANTHROPIC_API_KEY")
    gemini_key = os.getenv("GOOGLE_API_KEY")

    tasks = [
        call_openai(prompt, api_key=openai_key),
        call_claude(prompt, api_key=claude_key),
        call_gemini(prompt, api_key=gemini_key),
    ]

    raw_results = await asyncio.gather(*tasks)

    model_names = ["openai", "claude", "gemini"]
    results = {}

    for name, raw in zip(model_names, raw_results):
        validated = AnalysisOutput(**raw)
        results[name] = validated.model_dump()

    return results


In [10]:
async def test_parallel_initial(clause_text):
    start = time.time()
    result = await parallel_initial_analysis(clause_text)
    end = time.time()
    print("Parallel initial time:", round(end - start, 2), "seconds")
    return result


await test_parallel_initial(clauses[0]["clause_text"])


Parallel initial time: 11.67 seconds


{'openai': {'golden_clause_detected': True,
  'golden_clause_type': 'Payment',
  'risk_score': 3.0,
  'balanced': True,
  'justification': "The clause matches the 'Payment' type as it governs payment timelines and late fees. The risk is relatively low as it specifies clear payment terms and a reasonable interest rate for late payments.",
  'key_risk_indicators': ['undisputed invoices',
   '30 days of receipt',
   'interest at 2% per month']},
 'claude': {'golden_clause_detected': True,
  'golden_clause_type': 'Payment',
  'risk_score': 3.0,
  'balanced': True,
  'justification': "This clause matches the 'Payment' golden clause definition, which governs payment timelines and late fees. The clause specifies a 30-day payment timeline and a 2% monthly late fee, which is a standard and reasonable commercial term. The risk score is relatively low as the clause is balanced and does not contain any unusual or onerous provisions.",
  'key_risk_indicators': ['Late payment interest rate']},
 'gem

In [11]:
async def test_partial_parallel(clause_text):

    start = time.time()

    initial = await parallel_initial_analysis(clause_text)
    reviewed = await review_round(clause_text, initial)
    final = await arbitration(clause_text, reviewed)

    end = time.time()

    print("Partial parallel total time:", round(end - start, 2), "seconds")
    return final


await test_partial_parallel(clauses[0]["clause_text"])


Partial parallel total time: 35.16 seconds


{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.',
 'golden_clause_detected': True,
 'golden_clause_type': 'Payment',
 'final_risk_score': 5.0,
 'risk_level': 'Medium',
 'business_risk_if_ignored': "If the Client fails to pay invoices within the specified 30-day period, they will incur significant financial penalties. The 2% per month interest rate translates to a 24% annual percentage rate (APR), which is a substantially high late fee. This poses a considerable financial risk to the Client's cash flow and profitability in cases of delayed payments or disputes, potentially leading to substantial compounding debt.",
 'suggested_correction': "To promote a more balanced commercial relationship and mitigate excessive financial risk for the Client, the late payment interest rate should be revised. A more standard and commercially reasonable rate would be 1% to 1.5% per month (12% to 18% APR) or 

In [12]:
import asyncio
import os
import json
from models.openai_model import call_openai
from models.claude_model import call_claude
from models.gemini_model import call_gemini
from config.prompts import REVIEW_PROMPT
from core.schemas import ReviewOutput

async def parallel_review_round(clause_text, initial_outputs):

    openai_key = os.getenv("OPENAI_API_KEY")
    claude_key = os.getenv("ANTHROPIC_API_KEY")
    gemini_key = os.getenv("GOOGLE_API_KEY")

    model_names = ["openai", "claude", "gemini"]

    prompts = []

    for name in model_names:
        peers = {k: v for k, v in initial_outputs.items() if k != name}

        prompt = REVIEW_PROMPT.format(
            clause_text=clause_text,
            self_output=json.dumps(initial_outputs[name], indent=2),
            peer_outputs=json.dumps(peers, indent=2)
        )

        prompts.append(prompt)

    tasks = [
        call_openai(prompts[0], api_key=openai_key),
        call_claude(prompts[1], api_key=claude_key),
        call_gemini(prompts[2], api_key=gemini_key),
    ]

    raw_results = await asyncio.gather(*tasks)

    revised = {}

    for name, raw in zip(model_names, raw_results):
        validated = ReviewOutput(**raw)
        revised[name] = validated.model_dump()

    return revised


In [13]:
async def test_full_parallel(clause_text):

    start = time.time()

    initial = await parallel_initial_analysis(clause_text)
    reviewed = await parallel_review_round(clause_text, initial)
    final = await arbitration(clause_text, reviewed)

    end = time.time()

    print("Full parallel time:", round(end - start, 2), "seconds")
    return final


await test_full_parallel(clauses[0]["clause_text"])


Full parallel time: 33.69 seconds


{'clause_text': 'Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.',
 'golden_clause_detected': True,
 'golden_clause_type': 'Payment',
 'final_risk_score': 4.83,
 'risk_level': 'Medium',
 'business_risk_if_ignored': "The primary business risk for the Client is the significantly high late payment interest rate of 2% per month (24% per annum). If the Client faces any delays in payment, even for legitimate reasons or temporary cash flow issues, they will incur substantial and disproportionate penalties, leading to increased operational costs and potential financial strain. While the 'undisputed invoices' clause offers some protection, the high interest rate on late undisputed payments poses a significant financial liability.",
 'suggested_correction': "Revise the late payment interest rate of '2% per month' to a more commercially standard and balanced rate, such as '1% per month' (12% per annum) or a rate tied